In [11]:
!pip3 install transformers datasets pandas evaluate numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
label2id = {
 'AAA(RU)': 0,
 'AA(RU)': 1, 
 'A+(RU)': 2,
 'A(RU)': 3,
 'A-(RU)': 4,
 'BBB+(RU)': 5,
 'BBB(RU)': 6, 
 'AA+(RU)': 7,
 'BBB-(RU)': 8,
 'AA-(RU)': 9,
 'BB+(RU)': 10, 
 'BB-(RU)': 11, 
 'B+(RU)': 12,
 'BB(RU)': 13, 
 'B(RU)': 14,
 'B-(RU)': 15, 
 'C(RU)': 16
}
id2label = {0: 'AAA(RU)',
 1: 'AA(RU)',
 2: 'A+(RU)',
 3: 'A(RU)',
 4: 'A-(RU)',
 5: 'BBB+(RU)',
 6: 'BBB(RU)',
 7: 'AA+(RU)',
 8: 'BBB-(RU)',
 9: 'AA-(RU)',
 10: 'BB+(RU)',
 11: 'BB-(RU)',
 12: 'B+(RU)',
 13: 'BB(RU)',
 14: 'B(RU)',
 15: 'B-(RU)',
 16: 'C(RU)'}

In [2]:
import pickle
with open('dss.pickle', 'rb') as file:
    data = pickle.load(file)

In [3]:
data[0]

{'text': 'Кредитный рейтинг АКБ Энергобанк ПАО далее Энергобанк Банк обусловлен удовлетворительным бизнеспрофилем в сочетании с сильной достаточностью капитала критическим рискпрофилем и удовлетворительной оценкой ликвидности и фондирования учитывающей концентрацию обязательств Банка на средствах крупнейших кредиторовОсновная деятельность Энергобанка сконцентрирована в Республике Татарстан далее РТ где он занимает устойчивые рыночные позиции занял шестое место по размеру активов на На российском банковском рынке Банк имеет невысокую долю на он занимал е место по величине собственного капитала и е место по размеру активовКлючевыми направлениями деятельности Банка являются кредитование предприятий агропромышленного комплекса строительства и торговли а также залоговое розничное кредитование Контролирующими собственниками Банка являются ИН Хайруллин и АН Хайруллин владеющие около акций через АО Эдельвейс КорпорейшнОценка бизнеспрофиля Банка отражает его относительно невысокую долю на росси

In [5]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny", num_labels=len(id2label.keys()), id2label=id2label, label2id=label2id
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
def token(text):
    return tokenizer(text['text'], padding=True, truncation=True, max_length=512, return_tensors='pt')

In [7]:
from datasets import Dataset
import pandas as pd
from random import shuffle
shuffle(data)
train = data[:11500]
test = data[11500:]
train = Dataset.from_pandas(pd.DataFrame(data=train))
test = Dataset.from_pandas(pd.DataFrame(data=test))
tokenized_train = train.map(token, batched=True)
tokenized_test = test.map(token, batched=True)

Map: 100%|██████████| 1845/1845 [00:00<00:00, 3125.02 examples/s]


In [8]:
import evaluate
import numpy as np

f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

In [9]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
    output_dir="akra_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()